In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("../csv_files/1.19 Will FB 4.csv", encoding='latin-1')

In [ ]:
source = data["Destination"].value_counts().idxmax()
source

In [ ]:
data.iloc[0]

In [ ]:
data = data[~(data["Protocol"].isin(["MDNS", "ICMP", "DHCP", "NTP"])|data['Protocol'].isna())]
data["Protocol"].value_counts()

In [ ]:
# add new column host for ip host

def find_host(row):
    # find non-source ip address
    ip = str(row["Destination"] if row["Destination"] != source else row["Source"])

    # akamai
    if any(x in ip for x in ['google', 'gmail', '1e100']):
        return "Google"
    elif any(x in ip for x in ['amazon', 'aws']):
        return "Amazon"
    elif any(x in ip for x in ['apple-dns', 'aaplimg']):
        return "Apple"
    elif any(x in ip for x in ['akamai', 'akadns']):
        return "Akamai"
    elif any(x in ip for x in ['facebook', 'fbcdn']):
        return "Facebook"
    else:
        return "other"

data["Host"] = data.apply(find_host, axis=1)
data["Host"].value_counts()


In [ ]:
# if saved as seconds, convert to integer seconds
#data['Time'] = data['Time'].astype('int64')

# if saved as datetime, convert to integer seconds
times = pd.to_datetime(data['Time'])
data['Time'] = (times-times[0]).dt.seconds

# round seconds to nearest 5
# data['Time'] = data['Time'].apply(lambda x: round(x/5)*5)


In [ ]:
fig, ax = plt.subplots()

fb = data.where(data['Host'] == 'Facebook')['Length'].fillna(0)
goog = data.where(data['Host'] == 'Google')['Length'].fillna(0)
amz = data.where(data['Host'] == 'Amazon')['Length'].fillna(0)
aapl = data.where(data['Host'] == 'Apple')['Length'].fillna(0)
ak = data.where(data['Host'] == 'Akamai')['Length'].fillna(0)
oth = data.where(data['Host'] == 'other')['Length'].fillna(0)

size_data = pd.DataFrame({'Time': data['Time'], 'Facebook': fb, 'Google': goog, 'Amazon': amz, 'Akamai': ak, 'Apple': aapl, 'Other': oth})
size_data = size_data.groupby('Time', dropna=False).sum()

ax.stackplot(size_data.index, 
            size_data['Facebook'], size_data['Google'], size_data['Amazon'], size_data['Akamai'], size_data['Apple'], size_data['Other'], 
            labels=['Facebook', 'Google', 'Amazon', 'Akamai', 'Apple', 'Other'])

# ax.stackplot(data['Time'], fb, goog, amz, ak, labels=['Facebook', 'Google', 'Amazon', 'Akamai'])
ax.legend(loc='upper right')
ax.set_title('Packet length by host')
ax.set_xlabel('Second')
ax.set_ylabel('Total packet length')

plt.rcParams['figure.figsize'] = [15, 5]

plt.show()

In [ ]:
fb